In [1]:
import os
import re
from PyPDF2 import PdfReader
from openai import OpenAI  # Import selon votre instruction
from dotenv import load_dotenv
import json
from concurrent.futures import ThreadPoolExecutor
load_dotenv()
# Création de l'instance de client OpenAI avec votre clé API
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def find_pdf_in_project():
    for root, dirs, files in os.walk(os.getcwd()):
        for file in files:
            if file.endswith('.pdf'):
                return os.path.join(root, file)
    return None

def extraire_sections(pdf_path, output_dir="sections"):
    """Extrait les sections d'un PDF en Markdown en se basant sur les numéros de section seuls sur une ligne.

    Args:
      pdf_path: Chemin d'accès au fichier PDF.
      output_dir: Répertoire de sortie pour les fichiers Markdown.
    """
    os.makedirs(output_dir, exist_ok=True)
    reader = PdfReader(pdf_path)
    num_pages = len(reader.pages)

    section_texte = ""
    section_num = None
    section_metadata = []
    
    def process_section(file_path):
        analyser_texte(file_path)

    with ThreadPoolExecutor() as executor:
        for page_num in range(num_pages):
            try:
                page = reader.pages[page_num]
                texte = page.extract_text()

                if texte:
                    # Diviser le texte en lignes
                    lignes = texte.splitlines()
                    for ligne in lignes:
                        ligne = ligne.strip()
                        if ligne.isdigit():  # Vérifier si la ligne contient uniquement un chiffre
                            if section_num is not None:
                                # Enregistrer la section précédente
                                file_path = os.path.join(output_dir, f"{section_num}.md")
                                with open(file_path, "w", encoding="utf-8") as f:
                                    f.write(f"# Section {section_num}\n\n")
                                    f.write(section_texte.strip())
                                executor.submit(process_section, file_path)  # Passe le fichier par l'IA
                                # Ajouter les métadonnées de la section
                                section_metadata.append({
                                    'section_number': section_num,
                                    'file_path': file_path
                                })
                            section_num = int(ligne)
                            section_texte = f"# Section {section_num}\n\n"
                        else:
                            # Ajouter la ligne à la section courante
                            if section_num is not None:
                                section_texte += ligne + "\n"

            except Exception as e:
                print(f"Erreur lors du traitement de la page {page_num + 1}: {e}")

        # Sauvegarder la dernière section
        if section_num is not None:
            file_path = os.path.join(output_dir, f"{section_num}.md")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(f"# Section {section_num}\n\n")
                f.write(section_texte.strip())
            executor.submit(process_section, file_path)  # Passe le dernier fichier par l'IA
            # Ajouter les métadonnées de la dernière section
            section_metadata.append({
                'section_number': section_num,
                'file_path': file_path
            })

    # Sauvegarder les métadonnées dans un fichier JSON
    metadata_path = os.path.join(output_dir, 'section_metadata.json')
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(section_metadata, f, ensure_ascii=False, indent=4)
    print(f'Métadonnées sauvegardées dans: {metadata_path}')

def analyser_texte(file_path):
    """Utilise une IA pour analyser le texte et ajuster le formatage des numéros de chapitre."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            contenu = f.read()

        # Appel à l'API OpenAI en utilisant la méthode spécifiée
        completion = client.chat.completions.create(
            model="gpt-4o",  # ou le modèle de votre choix
            messages=[
                {"role": "system", "content": "You are a helpful assistant that formats text for gamebooks."},
                {"role": "user", "content": f"Here is a text extracted from a gamebook:\n\n{contenu}\n\nIdentify numbers representing chapters that the reader should go to and surround them with [[ ]]. Leave other numbers as they are. Don't Say : Certainly! Here's the formatted text: Or Something else than the text"}
            ]
        )

        texte_modifie = completion.choices[0].message.content

        # Enregistrer le texte modifié dans le fichier
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(texte_modifie)

    except Exception as e:
        print(f"Erreur lors de l'analyse du fichier {file_path}: {e}")

# Exemple d'utilisation
pdf_path = find_pdf_in_project()
if pdf_path:
    print(f'PDF trouvé: {pdf_path}')
    extraire_sections(pdf_path)
else:
    print('Aucun PDF trouvé dans le projet')
